In [12]:
import pandas as pd

def generer_tableau_resultats(fichier_excel):
    # Lire le fichier Excel
    df = pd.read_excel(fichier_excel)

    # Calculer les soldes des comptes
    soldes = df.groupby('Compte')[['Débit', 'Crédit']].sum()
    soldes['Solde'] = soldes['Débit'] - soldes['Crédit']

    # Réinitialiser l'index pour inclure 'Compte' dans le DataFrame
    soldes = soldes.reset_index()

    # Afficher le tableau de résultats
    print("Tableau de résultats :")
    print(soldes)

    return soldes,df


In [13]:

from common.path import DATA_PATH


fichier_excel = list(DATA_PATH.glob('2023*xls*'))[0]
soldes,df = generer_tableau_resultats(fichier_excel)



Tableau de résultats :
     Compte      Débit     Crédit      Solde
0    101300       0.00  153000.00 -153000.00
1    106100       0.00   15300.00  -15300.00
2    119000  264152.38  156249.73  107902.65
3    120000  156249.73  156249.73       0.00
4    164200  120000.00  240000.00 -120000.00
..      ...        ...        ...        ...
435  S00995     169.12     169.12       0.00
436  S00996      89.59      89.59       0.00
437  S00997     459.48     459.48       0.00
438  S00998     383.78     383.78       0.00
439  S00999      79.22      79.22       0.00

[440 rows x 4 columns]


In [14]:
df

,Compte,Intitulé,Date,Journal,Libellé,Rèf pièce,Débit,Let.,Crédit,Solde
0,101300,Cap.souscr. appele verse,01/01/2023,AN,REPRISE A NOUVEAU,NaN,0.00,NaN,153000.00,-153000.00
1,106100,Reserve legale,01/01/2023,AN,REPRISE A NOUVEAU,NaN,0.00,NaN,15300.00,-15300.00
2,119000,Report a nouveau debiteur,01/01/2023,AN,REPRISE A NOUVEAU,NaN,264152.38,NaN,0.00,264152.38
3,119000,Report a nouveau debiteur,30/06/2023,OD,affectation du resulatat,NaN,0.00,NaN,156249.73,107902.65
4,120000,Résultat de l'exercice (bénéfice),01/01/2023,AN,REPRISE A NOUVEAU,NaN,0.00,NaN,156249.73,-156249.73
...,...,...,...,...,...,...,...,...,...,...
39178,S00997,AVENIE FABRICE,31/12/2023,OD,OD REGUL ESP,NaN,274.82,AAAA,0.00,0.00
39179,S00998,ALVES KEVIN,31/07/2023,PA,ALVES KEVIN,NaN,0.00,AAAA,383.78,-383.78
39180,S00998,ALVES KEVIN,04/08/2023,CA IDF,VIREMENT WEB ALVES KEVIN SERVEUR SOLDE JUILLET...,NaN,383.78,AAAA,0.00,0.00
39181,S00999,ALI EBRAHIM,31/07/2023,PA,ALI EBRAHIM,NaN,0.00,AAAA,79.22,-79.22


In [15]:
df.columns

Index(['Compte', 'Intitulé', 'Date', 'Journal', 'Libellé', 'Rèf pièce',
       'Débit', 'Let.', 'Crédit', 'Solde'],
      dtype='object')

In [18]:
from pathlib import Path
import pandas as pd

def load_excel_data(path_list) -> pd.DataFrame:
    """Charge les données comptables depuis un fichier Excel."""
    df = pd.concat([pd.read_excel(p, dtype={'Compte': str}) for p in path_list])
    return df

def calculate_balance_sheet(df: pd.DataFrame) -> pd.DataFrame:
    """Calcule le bilan et retourne un DataFrame."""
    bilan = {
        "Catégorie": ["Actifs", "Passifs", "Capitaux Propres", "Total Bilan"],
        "Montant": [
            df[df['Compte'].str.startswith(('2', '3', '4'))]['Débit'].sum(),
            df[df['Compte'].str.startswith(('1', '5'))]['Crédit'].sum(),
            df[df['Compte'].str.startswith('1')]['Crédit'].sum(),
            df[df['Compte'].str.startswith(('2', '3', '4'))]['Débit'].sum() - df[df['Compte'].str.startswith(('1', '5'))]['Crédit'].sum()
        ]
    }
    return pd.DataFrame(bilan)

def calculate_income_statement(df: pd.DataFrame) -> pd.DataFrame:
    """Calcule le compte de résultat et retourne un DataFrame."""
    compte_resultat = {
        "Catégorie": ["Produits", "Charges", "Résultat Net"],
        "Montant": [
            df[df['Compte'].str.startswith('7')]['Crédit'].sum(),
            df[df['Compte'].str.startswith('6')]['Débit'].sum(),
            df[df['Compte'].str.startswith('7')]['Crédit'].sum() - df[df['Compte'].str.startswith('6')]['Débit'].sum()
        ]
    }
    return pd.DataFrame(compte_resultat)

def calculate_cash_flow_statement(df: pd.DataFrame) -> pd.DataFrame:
    """Calcule le tableau des flux de trésorerie et retourne un DataFrame."""
    flux = {
        "Catégorie": ["Flux Opérationnels", "Flux Investissements", "Flux Financements", "Variation de Trésorerie"],
        "Montant": [
            df[df['Compte'].str.startswith(('6', '7'))]['Débit'].sum() - df[df['Compte'].str.startswith(('6', '7'))]['Crédit'].sum(),
            df[df['Compte'].str.startswith('2')]['Débit'].sum() - df[df['Compte'].str.startswith('2')]['Crédit'].sum(),
            df[df['Compte'].str.startswith(('1', '5'))]['Crédit'].sum() - df[df['Compte'].str.startswith(('1', '5'))]['Débit'].sum(),
            (df[df['Compte'].str.startswith(('6', '7'))]['Débit'].sum() - df[df['Compte'].str.startswith(('6', '7'))]['Crédit'].sum()) +
            (df[df['Compte'].str.startswith('2')]['Débit'].sum() - df[df['Compte'].str.startswith('2')]['Crédit'].sum()) +
            (df[df['Compte'].str.startswith(('1', '5'))]['Crédit'].sum() - df[df['Compte'].str.startswith(('1', '5'))]['Débit'].sum())
        ]
    }
    return pd.DataFrame(flux)

def generate_beamer_presentation(bilan: pd.DataFrame, compte_resultat: pd.DataFrame, flux_tresorerie: pd.DataFrame, output_path: Path):
    """Génère une présentation Beamer en LaTeX avec les trois tableaux."""
    latex_template = r"""
\documentclass{beamer}
\usepackage{booktabs}

\begin{document}

\begin{frame}
    \frametitle{Bilan}
    \centering
    \begin{tabular}{lc}
        \toprule
        Catégorie & Montant \\
        \midrule
        %s
        \bottomrule
    \end{tabular}
\end{frame}

\begin{frame}
    \frametitle{Compte de Résultat}
    \centering
    \begin{tabular}{lc}
        \toprule
        Catégorie & Montant \\
        \midrule
        %s
        \bottomrule
    \end{tabular}
\end{frame}

\begin{frame}
    \frametitle{Tableau des Flux de Trésorerie}
    \centering
    \begin{tabular}{lc}
        \toprule
        Catégorie & Montant \\
        \midrule
        %s
        \bottomrule
    \end{tabular}
\end{frame}

\end{document}
    """

    def df_to_latex_rows(df):
        """Convertit un DataFrame en lignes LaTeX."""
        return "\n".join([f"{row['Catégorie']} & {row['Montant']:.0f} \\\\" for _, row in df.iterrows()])

    bilan_latex = df_to_latex_rows(bilan)
    compte_resultat_latex = df_to_latex_rows(compte_resultat)
    flux_tresorerie_latex = df_to_latex_rows(flux_tresorerie)

    latex_content = latex_template % (bilan_latex, compte_resultat_latex, flux_tresorerie_latex)

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(latex_content)

if __name__ == "__main__":
    excel_path = list(DATA_PATH.glob('202*xls*'))
    beamer_output_path = Path("presentation_beamer.tex")

    df = load_excel_data(excel_path)

    bilan_df = calculate_balance_sheet(df)
    compte_resultat_df = calculate_income_statement(df)
    flux_tresorerie_df = calculate_cash_flow_statement(df)

    print("📊 Bilan :\n", bilan_df)
    print("\n📈 Compte de Résultat :\n", compte_resultat_df)
    print("\n💰 Tableau de Flux de Trésorerie :\n", flux_tresorerie_df)

    generate_beamer_presentation(bilan_df, compte_resultat_df, flux_tresorerie_df, beamer_output_path)
    print(f"\n📄 Présentation Beamer générée : {beamer_output_path}")


ValueError: Cannot mask with non-boolean array containing NA / NaN values